In [41]:
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import shuffle
import numpy as np

In [42]:
df = pd.read_csv('NFL PBP.csv')

C:\Users\m224956\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [43]:
df.shape

(449371, 255)

In [44]:
df.head(10)

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
5,162,2009091000,PIT,TEN,TEN,away,PIT,TEN,98.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
6,183,2009091000,PIT,TEN,TEN,away,PIT,TEN,98.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
7,207,2009091000,PIT,TEN,TEN,away,PIT,TEN,94.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
8,228,2009091000,PIT,TEN,TEN,away,PIT,TEN,96.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
9,253,2009091000,PIT,TEN,PIT,home,TEN,TEN,43.0,2009-09-10,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


Pedicting "yards_gained" Given:
- game_seconds_remaining (from 3600 downwards)
- ydstogo (until a first down)
- down (the number, 1 to 4)
- shotgun (binary, yes or no)
- play_type (run or pass)
- qb_kneel or qb_spike (binary, yes or no)
- posteam_score and defteam_score (possessing and defensive)

In [45]:
nfl_data = df[['game_seconds_remaining', 
               'ydstogo',
               'down',
               'shotgun',
               'play_type',
               'qb_kneel',
               'qb_spike',
               'posteam_score',
               'defteam_score',
               'yards_gained']]
nfl_data.head()

,game_seconds_remaining,ydstogo,down,shotgun,play_type,qb_kneel,qb_spike,posteam_score,defteam_score,yards_gained
0,3600.0,0,NaN,0,kickoff,0,0,NaN,NaN,0.0
1,3593.0,10,1.0,0,pass,0,0,0.0,0.0,5.0
2,3556.0,5,2.0,0,run,0,0,0.0,0.0,-3.0
3,3515.0,8,3.0,1,pass,0,0,0.0,0.0,0.0
4,3507.0,8,4.0,0,punt,0,0,0.0,0.0,0.0


In [64]:
# generate a seed; make results reproducable
np.random.seed(896)

NFL = shuffle(nfl_data)
NFL.head()

,game_seconds_remaining,ydstogo,down,shotgun,play_type,qb_kneel,qb_spike,posteam_score,defteam_score,yards_gained
120673,2929.0,10,1.0,0,run,0,0,0.0,3.0,3.0
306270,2220.0,10,1.0,1,run,0,0,0.0,13.0,14.0
305825,914.0,10,1.0,0,pass,0,0,3.0,17.0,14.0
283066,40.0,4,2.0,0,run,0,0,27.0,30.0,-1.0
247443,2559.0,10,1.0,0,run,0,0,0.0,3.0,3.0


In [65]:
# feature, or input, dataframe
NFL_x  = NFL.drop('yards_gained',axis=1)
NFL_x.head()

,game_seconds_remaining,ydstogo,down,shotgun,play_type,qb_kneel,qb_spike,posteam_score,defteam_score
120673,2929.0,10,1.0,0,run,0,0,0.0,3.0
306270,2220.0,10,1.0,1,run,0,0,0.0,13.0
305825,914.0,10,1.0,0,pass,0,0,3.0,17.0
283066,40.0,4,2.0,0,run,0,0,27.0,30.0
247443,2559.0,10,1.0,0,run,0,0,0.0,3.0


In [66]:
# label, or output, dataframe
NFL_y = NFL['yards_gained']
NFL_y.head()

120673     3.0
306270    14.0
305825    14.0
283066    -1.0
247443     3.0
Name: yards_gained, dtype: float64

In [60]:
#split the data
valid_split = 0.2
test_split = 0.1

In [63]:
# create the splits in the dataframe
v_split = int(NFL.shape[0]*(1-valid_split-test_split))
t_split = int(NFL.shape[0]*(1-test_split))

# assign training, validation, and test data
X_train, Y_train  = NFL_x[:v_split], NFL_y[:v_split]
X_valid, Y_valid  = NFL_x[v_split:t_split], NFL_y[v_split:t_split]
X_test , Y_test   = NFL_x[t_split:], NFL_y[t_split:]

In [ ]:
#standardize the data; TODO
scaler = prep.StandardScaler().fit(X_train) # mean and variance calculated on X_train
X_train = scaler.transform(X_train) # based on the calculated mean and var X_train is scaled
X_valid = scaler.transform(X_valid) # based on the calculated mean and var X_valid is scaled
X_test  = scaler.transform(X_test)  # based on the calculated mean and var X_test is scaled